## J1939 Address Claim
J1939 has use cases where systems that utilize J1939 are reconfigured frequently. These secenarios include adding and removing construction or agriculture implemements or otherwise altering the vehicle network. In these situations, there is a risk of two controller applications (CAs) using the same source address (SA). Since messaging in J1939 includes a peer-to-peer mode with a source and destination address, there needs to be a mechanism to claim an address.

###  Technical Requirements 
SAE J1939-81 specifies some requirements:
1. Controller Applications must be capable of providing its unique 64-bit NAME.
2. CAs must claim an address before sending messages
3. Unsuccessful address claims must be handled and reported to the network.
4. A CA must follow the address claiming initialization.
5. A CA must support network managment requirements, including responses to power interruptions. 

#### The 64-bit NAME
SAE defines the following fields in a 64-bin NAME:

* 1 Bit : Arbitrary Address Capable
* 3 Bits: Industry Group (SAE)
* 4 Bits: Vehicle Systems Instance
* 7 Bits: Vehicle System (SAE)
* 1 Bit : Reserved (SAE)
* 8 Bits: Function (SAE)
* 5 Bits: Function Instance
* 3 Bits: ECU Instance
* 11 Bits: Manufacturer Code (SAE)
* 21 Bits: Identity

Some of these fields are specified by SAE in the J1939-81 standard; others are specified by manufacturers. The details of these are as follows:
1. Arbitrary Address Capable: Indicates if a CA can negotiate its address. If the bit is set, then it can negotiate. If the bit = 0, then it cannot, which means it will always want to keep its address.
2. Industry Group: These bits refer to the particular industy, like on-highway, construction, marine, etc.
3. Vehicle Systems Instance: a number for each instance on a vehicle system in case there are several networks (e.g. train cars).
4. Vehicle System: These are associated with industry groups and specify what kind of vehicle we are discussing, like  a tractor or a trailer.
5. Reserved: Specified to be zero.
6. Function: This is a code that if it is between 128 and 255, it would be associated with a particular industry group. If it is between 0 and 127, then it is not associated with other parameters.
7. ECU Instance: There may be multiple CAs of the same kind (i.e. multiple engines on a boat), so the intance distinguishes them. 
8. Manufacturer Code: This code is assigned by SAE to different manufacturers.
9. Identity Number: The is like a serial number assigned by the manufacturer.

For example,
![image-3.png](attachment:image-3.png)

The J1939DA_2020.xls spreadsheet (i.e. the Digital Annex) has tables for all the NAME fields.

In [2]:
import struct

### NAME Building
Let's build a name for a secure gateway device. This device may act as a gateway for the vehicle J1939 network to a telematics device to communicate with a back office server. 

In [36]:
# Start with the first byte
# 1 Bit : Arbitrary Address Capable = 1 (true)
# 3 Bits: Industry Group (SAE) = 1 (on-highway equipment)
# 4 Bits: Vehicle Systems Instance = 1 (the first instance)
arbitrary_address_capable_mask = 0b10000000
industry_group_mask = 0b01110000
veh_sys_instance_mask = 0b00001111

arbitrary_address_capable = 1
industry_group = 1
veh_sys_instance = 1

# Construct byte 8 according to J1939-81
NAME_8 = arbitrary_address_capable_mask & arbitrary_address_capable << 7
NAME_8 |= industry_group_mask & industry_group << 4
NAME_8 |= veh_sys_instance_mask & veh_sys_instance
print("NAME_8 = 0b{:08b}".format(NAME_8))
NAME = struct.pack('B',NAME_8)
NAME

NAME_8 = 0b10010001


b'\x91'

In [30]:
#masks show the coverage of bytes 
print("{:08b}".format(arbitrary_address_capable_mask))
print("{:08b}".format(industry_group_mask))
print("{:08b}".format(veh_sys_instance_mask))


10000000
01110000
00001111


## Vehicle Systems and Functions
The description of the vehicle systems depends on in industry group. There are 6 industry groups as follows:

| Industry Group ID | Industry Group Description |
| :-: | :-- |
| 0 | Global, applies to all |
| 1 | On-Highway Equipment |
| 2 | Agricultural and Forestry Equipment |
| 3 | Construction Equipment |
| 4 | Marine |
| 5 | Industrial-Process Control-Stationary (Gen-Sets) |

Based on the industry group, there are different systems. Many common systems are global and do not have a specific system. This means many NAME fields for the industrial group and system ID will be zero.

| Industry Group ID | Vehicle System ID | Vehicle System Description|
| :-: |  :-: |  :- |
| 0 | 0	 | Non-specific System |
| 1 | 0	 | Non-specific System |
| 1 | 1	 | Tractor |
| 1 | 2	 | Trailer |
| 2 | 0	 | Non-specific System |
| 2 | 1	 | Tractor |
| 2 | 2	 | Tillage |
| 2 | 3	 | Secondary Tillage |
| 2 | 4	 | Planters/Seeders |
| 2 | 5	 | Fertilizers |
| 2 | 6	 | Sprayers |
| 2 | 7	 | Harvesters |
| 2 | 8	 | Root Harvesters |
| 2 | 9	 | Forage |
| 2 | 10 | 	Irrigation |
| 2 | 11 | 	Transport/Trailer |
| 2 | 12 | 	Farm Yard Operations |
| 2 | 13 | 	Powered Auxiliary Devices |
| 2 | 14 | 	Special Crops |
| 2 | 15 | 	Earth Work |
| 2 | 16 | 	Skidder |
| 2 | 17 | 	Sensor Systems |
| 2 | 19 | 	Timber Harvesters |
| 2 | 20 | 	Forwarders |
| 2 | 21 | 	Timber Loaders |
| 2 | 22 | 	Timber Processing Machines |
| 2 | 23 | 	Mulchers |
| 2 | 24 | 	Utility Vehicles |
| 2 | 25 | 	Slurry/Manure Applicators |
| 2 | 26 | 	Feeders/Mixers |
| 2 | 27 | 	Weeders |


In [37]:
# Let's build Byte 7
# 7 Bits: Vehicle System (SAE) 
# 1 Bit : Reserved (SAE) = 0
vehicle_system_mask = 0b11111110
vehicle_system = 2 #Global, Non-specific System
NAME_7 = vehicle_system_mask & vehicle_system << 1
print("NAME_7 = 0b{:08b}".format(NAME_7))
NAME += struct.pack('B',NAME_7)
NAME

NAME_7 = 0b00000100


b'\x91\x04'

### All Industry Group Inclusive NAME Functions 
There are lists in the J1939 Digital Annex that describes the different functions. The main list is the Global Name Functions, and the second is the Industry Group Specific Name Functions.

Here is an example of a table entry in the global list

Function ID: 28

Function description: Off Vehicle Gateway

Notes: ECU for connecting between vehicle network(s) and an off-vehicle system or network, such as fleet management. Connection may be wireless. Performs Gateway functions, i.e., filters messages, translates between protocols. 

### Industry Group and Vehicle System Dependent NAME Functions 
Vehicle System is a 7-bit field defined by SAE J1939-81. It is combined with the
Industry Group to represent a common name. See J1939 (top-level document) Appendix B, Table
B12. Vehicle System provides a common name for a group of functions within a connected network.
Examples of Vehicle Systems for currently defined Industry Groups are "tractor" in the "Common” Industry Group, “Trailer” in the On-Highway Industry Group, and planter in the "Agricultural Equipment” Industry Group.



In [38]:
# Let's build Byte 6
# 8 Bits: Function (SAE) 

vehicle_function_mask = 0xFF
vehicle_function = 28 #Off Vehicle Gateway
NAME_6 = (vehicle_function_mask & vehicle_function)
print("NAME_6 = 0b{:08b}".format(NAME_6))
NAME += struct.pack('B',NAME_6)
NAME

NAME_6 = 0b00011100


b'\x91\x04\x1c'

In [39]:
# NAME Byte 5 has the following:
# 5 Bits: Function Instance
# 3 Bits: ECU Instance
function_instance_mask = 0b00011111
ecu_instance_mask = 0b11100000

function_instance = 1
ecu_instance = 1


# Construct byte 8 according to J1939-81
NAME_5 = ecu_instance_mask & ecu_instance << 5
NAME_5 |= function_instance_mask & function_instance
print("NAME_5 = 0b{:08b}".format(NAME_5))
NAME += struct.pack('B',NAME_5)
NAME

NAME_5 = 0b00100001


b'\x91\x04\x1c!'

In [54]:
# Now we can create a 32-bit work comprising the Manufacturer and their serial number.
# 11 Bits: Manufacturer Code (SAE)
# 21 Bits: Identity

manufacturer_code = 45 #This is made up and doesn't have an entry in the table.
manufacturer_code_mask = 0xFFE00000

identity = 123456
identity_mask = 0x001FFFFF

NAME_ID = manufacturer_code_mask & manufacturer_code << 21
NAME_ID |= identity_mask & identity 

print("ma_code = 0b{:032b}".format(manufacturer_code)) 
print("NAME_ID = 0b{:032b}".format(NAME_ID)) 
print("NAME_ID = 0x{:08X}".format(NAME_ID)) 
NAME += struct.pack('>L',NAME_ID)
NAME

ma_code = 0b00000000000000000000000000101101
NAME_ID = 0b00000101101000011110001001000000
NAME_ID = 0x05A1E240


b'\x91\x04\x1c!\xfa\x01\xe2@\x05\xa1\xe2@\x05\xa1\xe2@\x05\xa1\xe2@\x05\xa1\xe2@\x05\xa1\xe2@'

In [41]:
# Create a 64-bit NAME field
NAME_bytes = struct.pack("4B",NAME_8,NAME_7,NAME_6,NAME_5)
NAME_bytes += struct.pack(">L", NAME_ID)
print(NAME_bytes)
name_int = struct.unpack(">Q",NAME_bytes)[0]
print("NAME_bytes = {:08X}".format(name_int))
print("NAME_bytes = {:064b}".format(name_int))

b'\x91\x04\x1c!\xfa\x01\xe2@'
NAME_bytes = 91041C21FA01E240
NAME_bytes = 1001000100000100000111000010000111111010000000011110001001000000


### Preferred Address
For a quick address claiming process, ECUs and CAs should have a preferred address. 
The following are some preferred addresses:

*  0	Engine #1
*  1	Engine #2
*  2	Turbocharger
*  3	Transmission #1
*  4	Transmission #2
*  5	Shift Console - Primary
*  6	Shift Console - Secondary
*  7	Power TakeOff - (Main or Rear)
*  8	Axle - Steering
*  9	Axle - Drive #1
* 10	Axle - Drive #2
* 11	Brakes - System Controller
* 12	Brakes - Steer Axle
* 13	Brakes - Drive axle #1
* 14	Brakes - Drive Axle #2
* 15	Retarder - Engine
* 16	Retarder - Driveline
* 17	Cruise Control
* 18	Fuel System
* 19	Steering Controller
* 20	Suspension - Steer Axle
* 21	Suspension - Drive Axle #1
* 22	Suspension - Drive Axle #2
* 23	Instrument Cluster #1
* 24	Trip Recorder
* 25	Passenger-Operator Climate Control #1
* 26	Alternator/Electrical Charging System
* 27	Aerodynamic Control
* 28	Vehicle Navigation
* 29	Vehicle Security
* 30	Electrical System
* 31	Starter System
* 32	Tractor-Trailer Bridge #1
* 33	Body Controller
* 34	Auxiliary Valve Control or Engine Air System Valve Control
* 35	Hitch Control
* 36	Power TakeOff (Front or Secondary)
* 37	Off Vehicle Gateway
* 38	Virtual Terminal (in cab)
* 39	Management Computer #1
* 40	Cab Display #1
* 41	Retarder, Exhaust, Engine #1
* 42	Headway Controller
* 43	On-Board Diagnostic Unit
* 44	Retarder, Exhaust, Engine #2
* 45	Endurance Braking System
* 46	Hydraulic Pump Controller
* 47	Suspension - System Controller #1
* 48	Pneumatic - System Controller
* 49	Cab Controller - Primary
* 50	Cab Controller - Secondary
* 51	Tire Pressure Controller
* 52	Ignition Control Module #1
* 53	Ignition Control Module #2
* 54	Seat Control #1
* 55	Lighting - Operator Controls
* 56	Rear Axle Steering Controller #1
* 57	Water Pump Controller
* 58	Passenger-Operator Climate Control #2
* 59	Transmission Display - Primary
* 60	Transmission Display - Secondary
* 61	Exhaust Emission Controller
* 62	Vehicle Dynamic Stability Controller
* 63	Oil Sensor
* 64	Suspension - System Controller #2
* 65	Information System Controller #1
* 66	Ramp Control
* 67	Clutch/Converter Unit
* 68	Auxiliary Heater #1
* 69	Auxiliary Heater #2
* 70	Engine Valve Controller
* 71	Chassis Controller #1
* 72	Chassis Controller #2
* 73	Propulsion Battery Charger
* 74	Communications Unit, Cellular
* 75	Communications Unit, Satellite
* 76	Communications Unit, Radio
* 77	Steering Column Unit
* 78	Fan Drive Controller
* 79	Seat Control #2
* 80	Parking Brake Controller
* 81	Aftertreatment #1 system gas intake
* 82	Aftertreatment #1 system gas outlet
* 83	Safety Restraint System
* 84	Cab Display #2
* 85	Diesel Particulate Filter Controller
* 86	Aftertreatment #2 system gas intake
* 87	Aftertreatment #2 system gas outlet
* 88	Safety Restraint System #2
* 89	Atmospheric Sensor
* 90	Powertrain Control Module
* 91	Power Systems Manager
* 92	Engine Injection Control Module
* 93	Fire Protection System
* 94	Driver Impairment Device

* 128	thru 247	Used for dynamic address assignment

* 248	File Server / Printer
* 249	Off Board Diagnostic-Service Tool #1
* 250	Off Board Diagnostic-Service Tool #2
* 251	On-Board Data Logger

* 254	Null Address
* 255	GLOBAL (All-Any Node)


## Address Claim Procedure
The following procedure was adapted from SAE J1939 ECU Programming and Vehicle Bus Simulation by Wilfried Voss

![AddressClaimProcedure.svg](AddressClaimProcedure.svg)

### Looking for an Address Claim
During startup of an ECU on J1939, an Address Claimed Message will often be broadcast. Here's an example.

The following command was given from a Linux terminal 
```
candump any | grep 18EE
```
That gives the following output
```
  can1  18EEFF00   [8]  06 03 BF 01 00 00 00 10
  can1  18EEFF00   [8]  06 03 BF 01 00 00 00 10
  can1  18EEFF00   [8]  06 03 BF 01 00 00 00 10
```
To interpret this message, we start with breaking down the ID. Let's break it into 4 parts: 18 EE FF 00

* Priority: 6 (0x18 >> 2)
* PGN: 0xEE00 = 60928 (Address Claim)
* Destination Address: 0xFF = 255 (Global)
* Source Address: 0x00 (Engine #1)

Next, we decode the different bytes from the message, which makes up the NAME field.
The following table shows the position and interpretation of the data in NAME according to J1939-81.

| Byte | Bits | Description | Value |
| --- | --- | --- | --- | 
| Byte 1 | Bits 8-1 | Least significant byte of Identity Number | 0x06 |
| Byte 2 | Bits 8-1 | Second byte of Identity Number |0x03|
| Byte 3 | Bits 8-6 | Least significant 3 bits of Manufacturer Code | 0xBF |
| Byte 3 | Bits 5-1 | Most significant 5 bits of Identity Number | 0xBF  |
| Byte 4 | Bits 8-1 | Most significant 8 bits of Manufacturer Code | 0x01 |
| Byte 5 | Bits 8-4 | Function Instance | 0x00 |
| Byte 5 | Bits 3-1 | ECU Instance | 0x00 |
| Byte 6 | Bits 8-1 | Function  | 0x00 |
| Byte 7 | Bits 8-2 | Vehicle System  | 0x00 |
| Byte 7 | Bit 1 | Reserved | 0x00 |
| Byte 8 | Bit 8 | Arbitrary Address Capable | 0x10 |
| Byte 8 | Bits 7-5 | Industry Group | 0x10 |
| Byte 8 | Bits 4-1 | Vehicle System Instance | 0x10 |

Let's parse these a little better.



In [73]:
name_bytes = bytes.fromhex('06 03 BF 01 00 00 00 10')
#name_bytes = bytes.fromhex('00 00 60 01 00 81 00 00')
name_bytes

b'\x06\x03\xbf\x01\x00\x00\x00\x10'

In [74]:
NAME_8 = name_bytes[7]

arbitrary_address_capable = (arbitrary_address_capable_mask & NAME_8 ) >> 7
print('arbitrary_address_capable =', arbitrary_address_capable)

industry_group = (industry_group_mask & NAME_8) >> 4
print('industry_group =', industry_group)

veh_sys_instance = (veh_sys_instance_mask & NAME_8)
print('veh_sys_instance =', veh_sys_instance)

arbitrary_address_capable = 0
industry_group = 1
veh_sys_instance = 0


The industry group is 1, which is the On-Highway Equipment group.

Bytes 7, 6 and 5 are all zeros, which indicates nonspecific vehicle system IDs, and the first instance of the Engine.

In [78]:
NAME_ID = struct.unpack("<L",name_bytes[0:4])[0]
manufacturer_code = (manufacturer_code_mask & NAME_ID) >> 21
print("NAME_ID = 0b{:032b}".format(NAME_ID)) 
print('manufacturer_code =', manufacturer_code)

serial_num = identity_mask & NAME_ID
print('serial_num = ',serial_num)


NAME_ID = 0b00000001101111110000001100000110
manufacturer_code = 13
serial_num =  2032390


The manufacture for 13 is Delco Electronics. The ECU we were connected to was made by Delphi, which used to be Delco.

### Send an Address Claimed Message
The J1939 address claimed message has a pgn of 60928 (0xEE00). 

Let's craft a message and send it out on the network. We'll try to claim source address 0, which is an engine controller.

In [79]:
import socket
import struct

# Open a socket and bind to it from SocketCAN
sock = socket.socket(socket.PF_CAN, socket.SOCK_RAW, socket.CAN_RAW)

#Change this interface to match your desired connection
interface = "can1"

# Bind to the interface
sock.bind((interface,))

# To match the socketCAN data structure, the following struct format can be used:
can_frame_format = "<lB3x8s"

AttributeError: module 'socket' has no attribute 'PF_CAN'

In [80]:
# Let's send the same message that we recieved from the engine controller to 
# see what happens
can_id = 0x18EEFF00

#Set the extended frame format bit.
can_id |= socket.CAN_EFF_FLAG
    
can_data = name_bytes

can_dlc = min(len(can_data),8)

can_packet = struct.pack(can_frame_format, can_id, can_dlc, can_data[:can_dlc] )
print(can_packet)
    
#Send out the CAN frame
sock.send(can_packet)

AttributeError: module 'socket' has no attribute 'CAN_EFF_FLAG'